# Denial of Service (DoS) Attacks - Hands-On Lab

**Part of HackLearn Pro - Module #18**

This interactive notebook provides hands-on exercises for understanding DoS/DDoS attacks, detection techniques, and defense mechanisms.

**Learning Objectives:**
- Implement token bucket rate limiting
- Detect HTTP flood attacks from logs
- Calculate DNS amplification factors
- Configure SYN flood defenses
- Set up rate limiting in web servers
- Analyze DDoS traffic patterns
- Configure CDN protection

**WARNING:** These techniques are for educational and authorized testing purposes only. Unauthorized DoS attacks are illegal under CFAA (18 USC § 1030) and similar laws worldwide.

## Lab 1: Token Bucket Rate Limiting Implementation

Token bucket is a common algorithm for rate limiting that allows bursts while maintaining average rate limits.

In [ ]:
import time
from collections import defaultdict
from datetime import datetime

class TokenBucket:
    """Token bucket rate limiter implementation."""
    
    def __init__(self, capacity, refill_rate):
        """
        Args:
            capacity: Maximum number of tokens (burst size)
            refill_rate: Tokens added per second
        """
        self.capacity = capacity
        self.refill_rate = refill_rate
        self.tokens = capacity
        self.last_refill = time.time()
    
    def consume(self, tokens=1):
        """Attempt to consume tokens. Returns True if allowed."""
        self._refill()
        
        if self.tokens >= tokens:
            self.tokens -= tokens
            return True
        return False
    
    def _refill(self):
        """Refill tokens based on elapsed time."""
        now = time.time()
        elapsed = now - self.last_refill
        
        # Add tokens based on elapsed time
        tokens_to_add = elapsed * self.refill_rate
        self.tokens = min(self.capacity, self.tokens + tokens_to_add)
        self.last_refill = now

class RateLimiter:
    """Per-IP rate limiter using token buckets."""
    
    def __init__(self, requests_per_second=10, burst_size=20):
        self.buckets = defaultdict(
            lambda: TokenBucket(burst_size, requests_per_second)
        )
    
    def allow_request(self, ip_address):
        """Check if request from IP should be allowed."""
        return self.buckets[ip_address].consume()

# Test the rate limiter
limiter = RateLimiter(requests_per_second=5, burst_size=10)

print("Testing rate limiter with 5 req/s, burst of 10:")
print()

# Simulate burst of requests
test_ip = "192.168.1.100"
for i in range(15):
    allowed = limiter.allow_request(test_ip)
    status = "ALLOWED" if allowed else "BLOCKED"
    print(f"Request {i+1}: {status}")

print("\nWaiting 1 second for token refill...")
time.sleep(1)

print("\nAfter 1 second (should have ~5 new tokens):")
for i in range(7):
    allowed = limiter.allow_request(test_ip)
    status = "ALLOWED" if allowed else "BLOCKED"
    print(f"Request {i+1}: {status}")

**Key Observations:**
- First 10 requests allowed (burst capacity)
- Additional requests blocked
- After 1 second, approximately 5 new tokens available
- Protects against sudden flood while allowing legitimate bursts

## Lab 2: HTTP Flood Detection from Access Logs

Analyze web server logs to detect potential HTTP flood attacks by identifying abnormal request patterns.

In [ ]:
import re
from collections import Counter, defaultdict
from datetime import datetime, timedelta

# Sample Apache/Nginx access log entries
sample_logs = """
192.168.1.100 - - [28/Oct/2025:10:15:01 +0000] "GET /api/products HTTP/1.1" 200 1234
192.168.1.100 - - [28/Oct/2025:10:15:01 +0000] "GET /api/products HTTP/1.1" 200 1234
192.168.1.100 - - [28/Oct/2025:10:15:01 +0000] "GET /api/products HTTP/1.1" 200 1234
192.168.1.101 - - [28/Oct/2025:10:15:02 +0000] "GET /index.html HTTP/1.1" 200 5678
192.168.1.100 - - [28/Oct/2025:10:15:02 +0000] "GET /api/products HTTP/1.1" 200 1234
192.168.1.100 - - [28/Oct/2025:10:15:02 +0000] "GET /api/products HTTP/1.1" 200 1234
10.0.0.50 - - [28/Oct/2025:10:15:03 +0000] "POST /login HTTP/1.1" 200 890
192.168.1.100 - - [28/Oct/2025:10:15:03 +0000] "GET /api/products HTTP/1.1" 200 1234
192.168.1.100 - - [28/Oct/2025:10:15:03 +0000] "GET /api/products HTTP/1.1" 200 1234
192.168.1.100 - - [28/Oct/2025:10:15:03 +0000] "GET /api/products HTTP/1.1" 200 1234
192.168.1.102 - - [28/Oct/2025:10:15:04 +0000] "GET /about.html HTTP/1.1" 200 2345
192.168.1.100 - - [28/Oct/2025:10:15:04 +0000] "GET /api/products HTTP/1.1" 200 1234
192.168.1.100 - - [28/Oct/2025:10:15:04 +0000] "GET /api/products HTTP/1.1" 200 1234
""".strip().split('\n')

def parse_log_line(line):
    """Parse Apache/Nginx log line."""
    pattern = r'^(\S+) .* \[([^\]]+)\] "(\S+) (\S+) \S+" (\d+) (\d+)'
    match = re.match(pattern, line)
    
    if match:
        return {
            'ip': match.group(1),
            'timestamp': match.group(2),
            'method': match.group(3),
            'path': match.group(4),
            'status': int(match.group(5)),
            'size': int(match.group(6))
        }
    return None

def detect_http_flood(logs, threshold=10, time_window=5):
    """Detect potential HTTP flood attacks."""
    # Parse all log entries
    parsed_logs = [parse_log_line(log) for log in logs]
    parsed_logs = [log for log in parsed_logs if log]
    
    # Count requests per IP
    ip_requests = Counter(log['ip'] for log in parsed_logs)
    
    # Count requests per IP per path
    ip_path_requests = defaultdict(Counter)
    for log in parsed_logs:
        ip_path_requests[log['ip']][log['path']] += 1
    
    # Detect suspicious IPs
    suspicious_ips = []
    
    for ip, count in ip_requests.items():
        if count >= threshold:
            # Check for repetitive patterns
            paths = ip_path_requests[ip]
            most_common_path = paths.most_common(1)[0]
            
            # If >80% of requests are to same endpoint, likely flood
            if most_common_path[1] / count > 0.8:
                suspicious_ips.append({
                    'ip': ip,
                    'total_requests': count,
                    'most_requested_path': most_common_path[0],
                    'path_requests': most_common_path[1],
                    'repetition_rate': f"{(most_common_path[1] / count * 100):.1f}%"
                })
    
    return suspicious_ips

# Analyze the logs
print("HTTP Flood Detection Analysis")
print("=" * 60)
print()

suspicious = detect_http_flood(sample_logs, threshold=8)

if suspicious:
    print(f"ALERT: Detected {len(suspicious)} suspicious IP(s):\n")
    for attacker in suspicious:
        print(f"IP Address: {attacker['ip']}")
        print(f"Total Requests: {attacker['total_requests']}")
        print(f"Targeted Path: {attacker['most_requested_path']}")
        print(f"Repetition Rate: {attacker['repetition_rate']}")
        print(f"Status: POTENTIAL HTTP FLOOD")
        print()
else:
    print("No suspicious activity detected.")

# Show request distribution
print("Request Distribution by IP:")
print("-" * 40)
ip_counts = Counter()
for log in sample_logs:
    parsed = parse_log_line(log)
    if parsed:
        ip_counts[parsed['ip']] += 1

for ip, count in ip_counts.most_common():
    print(f"{ip}: {count} requests")

**Detection Indicators:**
- High request volume from single IP
- Repetitive requests to same endpoint (>80%)
- Uniform timing patterns
- Lack of diversity in user agent strings (not shown)
- Absence of typical browsing behavior

## Lab 3: DNS Amplification Factor Calculator

Calculate amplification factors for DNS reflection attacks to understand their devastating potential.

In [ ]:
def calculate_amplification_factor(query_size, response_size):
    """Calculate DNS amplification factor."""
    return response_size / query_size

# Common DNS amplification scenarios
amplification_scenarios = [
    {
        'name': 'Basic A Record Query',
        'query_size': 60,  # bytes
        'response_size': 100,
        'description': 'Simple domain lookup'
    },
    {
        'name': 'ANY Record Query (Legacy)',
        'query_size': 60,
        'response_size': 4000,
        'description': 'Returns all DNS records (often blocked now)'
    },
    {
        'name': 'DNSSEC-signed Response',
        'query_size': 44,
        'response_size': 3223,
        'description': 'Query with DNSSEC signatures'
    },
    {
        'name': 'TXT Record with SPF',
        'query_size': 60,
        'response_size': 512,
        'description': 'SPF records can be verbose'
    },
    {
        'name': 'Large DNSKEY Response',
        'query_size': 44,
        'response_size': 1500,
        'description': 'DNSSEC public keys'
    }
]

print("DNS Amplification Factor Analysis")
print("=" * 80)
print()

for scenario in amplification_scenarios:
    factor = calculate_amplification_factor(
        scenario['query_size'],
        scenario['response_size']
    )
    
    print(f"Scenario: {scenario['name']}")
    print(f"Description: {scenario['description']}")
    print(f"Query Size: {scenario['query_size']} bytes")
    print(f"Response Size: {scenario['response_size']} bytes")
    print(f"Amplification Factor: {factor:.2f}x")
    print()

# Calculate attack bandwidth
print("\nAttack Bandwidth Calculation")
print("=" * 80)
print()

attacker_bandwidth_mbps = 10  # Attacker's upstream bandwidth
worst_case_scenario = amplification_scenarios[2]  # DNSSEC scenario
worst_factor = calculate_amplification_factor(
    worst_case_scenario['query_size'],
    worst_case_scenario['response_size']
)

amplified_bandwidth = attacker_bandwidth_mbps * worst_factor

print(f"Attacker's Bandwidth: {attacker_bandwidth_mbps} Mbps")
print(f"Using: {worst_case_scenario['name']}")
print(f"Amplification Factor: {worst_factor:.2f}x")
print(f"Resulting Attack Bandwidth: {amplified_bandwidth:.2f} Mbps")
print()
print(f"Impact: An attacker with just {attacker_bandwidth_mbps} Mbps can generate")
print(f"{amplified_bandwidth:.2f} Mbps of attack traffic toward the victim!")

# Calculate for different attacker bandwidths
print("\nScaled Attack Scenarios:")
print("-" * 60)
for bw in [1, 10, 100, 1000]:
    amplified = bw * worst_factor
    print(f"{bw:>4} Mbps attacker → {amplified:>8.2f} Mbps attack ({amplified/1000:.3f} Gbps)")

**Key Insights:**
- DNSSEC queries provide 70-80x amplification
- ANY queries historically provided up to 100x (now often blocked)
- Small botnets can generate massive attack traffic
- Modern defenses: Rate limiting, response rate limiting (RRL), BCP38

## Lab 4: SYN Flood Defense Configuration

Configure Linux kernel parameters and iptables rules to defend against SYN flood attacks.

In [ ]:
# This lab demonstrates SYN flood defense configurations
# In a real environment, these would be applied via sysctl and iptables

syn_flood_defenses = {
    'kernel_parameters': [
        {
            'parameter': 'net.ipv4.tcp_syncookies',
            'value': 1,
            'description': 'Enable SYN cookies to handle floods without filling backlog',
            'command': 'sysctl -w net.ipv4.tcp_syncookies=1'
        },
        {
            'parameter': 'net.ipv4.tcp_max_syn_backlog',
            'value': 4096,
            'description': 'Increase SYN queue size (default: 128)',
            'command': 'sysctl -w net.ipv4.tcp_max_syn_backlog=4096'
        },
        {
            'parameter': 'net.ipv4.tcp_synack_retries',
            'value': 2,
            'description': 'Reduce SYN-ACK retries (default: 5)',
            'command': 'sysctl -w net.ipv4.tcp_synack_retries=2'
        },
        {
            'parameter': 'net.core.netdev_max_backlog',
            'value': 5000,
            'description': 'Increase network device backlog',
            'command': 'sysctl -w net.core.netdev_max_backlog=5000'
        },
        {
            'parameter': 'net.ipv4.tcp_fin_timeout',
            'value': 15,
            'description': 'Reduce FIN-WAIT-2 timeout (default: 60)',
            'command': 'sysctl -w net.ipv4.tcp_fin_timeout=15'
        }
    ],
    'iptables_rules': [
        {
            'rule': 'Limit SYN packets per second per IP',
            'command': 'iptables -A INPUT -p tcp --syn -m limit --limit 1/s --limit-burst 3 -j ACCEPT'
        },
        {
            'rule': 'Drop excessive SYN packets',
            'command': 'iptables -A INPUT -p tcp --syn -j DROP'
        },
        {
            'rule': 'Protect against port scanning',
            'command': 'iptables -N port-scan'
        },
        {
            'rule': 'Log port scans',
            'command': 'iptables -A port-scan -j LOG --log-prefix "Port Scan: "'
        },
        {
            'rule': 'Drop port scan packets',
            'command': 'iptables -A port-scan -j DROP'
        }
    ],
    'monitoring': [
        {
            'command': 'netstat -an | grep SYN_RECV | wc -l',
            'description': 'Count current SYN_RECV connections'
        },
        {
            'command': 'ss -tan state syn-recv | wc -l',
            'description': 'Alternative using ss command'
        },
        {
            'command': 'watch -n 1 "netstat -an | grep SYN_RECV | wc -l"',
            'description': 'Real-time monitoring of SYN queue'
        }
    ]
}

print("SYN Flood Defense Configuration Guide")
print("=" * 80)
print()

print("KERNEL PARAMETERS (sysctl)")
print("-" * 80)
for param in syn_flood_defenses['kernel_parameters']:
    print(f"\nParameter: {param['parameter']}")
    print(f"Recommended Value: {param['value']}")
    print(f"Purpose: {param['description']}")
    print(f"Command: {param['command']}")

print("\n" + "=" * 80)
print("IPTABLES FIREWALL RULES")
print("-" * 80)
for rule in syn_flood_defenses['iptables_rules']:
    print(f"\nRule: {rule['rule']}")
    print(f"Command: {rule['command']}")

print("\n" + "=" * 80)
print("MONITORING COMMANDS")
print("-" * 80)
for mon in syn_flood_defenses['monitoring']:
    print(f"\nPurpose: {mon['description']}")
    print(f"Command: {mon['command']}")

print("\n" + "=" * 80)
print("\nPERSISTENT CONFIGURATION")
print("-" * 80)
print("\nTo make sysctl changes persistent, add to /etc/sysctl.conf:")
print()
for param in syn_flood_defenses['kernel_parameters']:
    print(f"{param['parameter']} = {param['value']}")
print("\nThen run: sysctl -p")

print("\nTo persist iptables rules:")
print("Debian/Ubuntu: iptables-save > /etc/iptables/rules.v4")
print("RHEL/CentOS: service iptables save")

**Defense Strategy:**
1. **SYN Cookies**: Allow handling connections without state until ACK received
2. **Increased Backlog**: Handle burst traffic without dropping legitimate connections
3. **Reduced Timeouts**: Free resources faster from incomplete handshakes
4. **Rate Limiting**: Limit new connections per IP per second
5. **Monitoring**: Real-time visibility into SYN queue depth

## Lab 5: Nginx Rate Limiting Configuration

Implement application-layer rate limiting using Nginx to protect against HTTP floods.

In [ ]:
# Nginx configuration examples for rate limiting

nginx_configs = {
    'basic_rate_limit': '''
# nginx.conf - HTTP context
http {
    # Define rate limit zone: 10MB stores ~160k IPs
    limit_req_zone $binary_remote_addr zone=general:10m rate=10r/s;
    
    server {
        listen 80;
        server_name example.com;
        
        location / {
            # Apply rate limit: 10 requests/second, burst of 20
            limit_req zone=general burst=20 nodelay;
            
            proxy_pass http://backend;
        }
    }
}
''',
    'api_protection': '''
# nginx.conf - Protect API endpoints
http {
    # Different zones for different endpoints
    limit_req_zone $binary_remote_addr zone=api:10m rate=5r/s;
    limit_req_zone $binary_remote_addr zone=login:10m rate=1r/s;
    limit_req_zone $binary_remote_addr zone=search:10m rate=20r/s;
    
    server {
        listen 80;
        server_name api.example.com;
        
        # Strict rate limit for login endpoint
        location /api/login {
            limit_req zone=login burst=3 nodelay;
            limit_req_status 429;  # Return 429 Too Many Requests
            
            proxy_pass http://backend;
        }
        
        # Moderate rate limit for general API
        location /api/ {
            limit_req zone=api burst=10 nodelay;
            limit_req_status 429;
            
            proxy_pass http://backend;
        }
        
        # Higher rate limit for search (read-only)
        location /api/search {
            limit_req zone=search burst=40 nodelay;
            limit_req_status 429;
            
            proxy_pass http://backend;
        }
    }
}
''',
    'connection_limits': '''
# nginx.conf - Limit concurrent connections
http {
    # Limit concurrent connections per IP
    limit_conn_zone $binary_remote_addr zone=addr:10m;
    
    server {
        listen 80;
        server_name example.com;
        
        # Max 10 concurrent connections per IP
        limit_conn addr 10;
        
        # Limit bandwidth per connection
        limit_rate 500k;  # 500 KB/s per connection
        
        location / {
            proxy_pass http://backend;
        }
    }
}
''',
    'advanced_protection': '''
# nginx.conf - Advanced DDoS protection
http {
    # Rate limiting zones
    limit_req_zone $binary_remote_addr zone=general:10m rate=10r/s;
    
    # Connection limiting
    limit_conn_zone $binary_remote_addr zone=addr:10m;
    
    # Geo-blocking (example: block specific countries)
    geo $geo_whitelist {
        default 1;  # Block by default
        1.2.3.0/24 0;  # Allow specific ranges
        10.0.0.0/8 0;  # Allow internal IPs
    }
    
    # Map to create custom block list
    map $http_user_agent $bad_bot {
        default 0;
        ~*curl 1;
        ~*wget 1;
        ~*python 1;
        ~*scanner 1;
    }
    
    server {
        listen 80;
        server_name example.com;
        
        # Block bad user agents
        if ($bad_bot) {
            return 403;
        }
        
        # Apply geo-blocking (if enabled)
        # if ($geo_whitelist) {
        #     return 403;
        # }
        
        location / {
            # Combined rate and connection limiting
            limit_req zone=general burst=20 nodelay;
            limit_conn addr 10;
            
            # Add security headers
            add_header X-RateLimit-Limit "10" always;
            add_header X-RateLimit-Remaining "$limit_req_status" always;
            
            proxy_pass http://backend;
            
            # Timeout settings
            proxy_connect_timeout 5s;
            proxy_send_timeout 10s;
            proxy_read_timeout 10s;
        }
    }
}
'''
}

print("Nginx Rate Limiting Configuration Guide")
print("=" * 80)

for config_name, config_content in nginx_configs.items():
    print(f"\n{'='*80}")
    print(f"Configuration: {config_name.replace('_', ' ').title()}")
    print(f"{'='*80}")
    print(config_content)

print("\n" + "=" * 80)
print("TESTING RATE LIMITS")
print("-" * 80)
print('''
# Test rate limiting with curl
for i in {1..100}; do
    curl -s -o /dev/null -w "Request $i: %{http_code}\\n" http://example.com/api/login
    sleep 0.1
done

# Expected output:
# Request 1-3: 200 (within burst limit)
# Request 4+: 429 (rate limited)

# Monitor Nginx rate limiting in real-time
tail -f /var/log/nginx/error.log | grep limiting
''')

print("\n" + "=" * 80)
print("MONITORING AND TUNING")
print("-" * 80)
print('''
# Check rate limit zone memory usage
# Add to nginx config:
location /nginx_status {
    stub_status on;
    access_log off;
    allow 127.0.0.1;
    deny all;
}

# View statistics
curl http://localhost/nginx_status

# Tune based on legitimate traffic patterns:
# 1. Monitor 429 responses: too many = limits too strict
# 2. Check error logs for "limiting requests" messages
# 3. Adjust rate and burst values accordingly
# 4. Consider whitelist for known good IPs
''')

**Key Nginx Rate Limiting Concepts:**

1. **Rate**: Average requests per second allowed (e.g., 10r/s)
2. **Burst**: Temporary excess allowed before rate limiting kicks in
3. **Nodelay**: Process burst requests immediately (don't queue)
4. **Zones**: Memory areas to track client states (10m = ~160k IPs)
5. **Status Code**: Return 429 (Too Many Requests) or 503 (Service Unavailable)

## Lab 6: DDoS Traffic Pattern Analysis

Analyze network traffic patterns to distinguish between legitimate spikes and DDoS attacks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Simulate traffic patterns
np.random.seed(42)

# Time series (24 hours, 1-minute intervals)
minutes = np.arange(0, 24*60, 1)

# Normal traffic pattern (follows daily cycle)
baseline = 100 + 50 * np.sin(2 * np.pi * minutes / (24*60) - np.pi/2)  # Daily cycle
normal_traffic = baseline + np.random.normal(0, 10, len(minutes))  # Add noise

# Simulate DDoS attack (sudden spike)
attack_start = 10 * 60  # 10:00 AM
attack_duration = 30  # 30 minutes
ddos_traffic = normal_traffic.copy()
ddos_traffic[attack_start:attack_start+attack_duration] += 500  # Massive spike

# Simulate legitimate traffic spike (gradual, follows pattern)
spike_start = 18 * 60  # 6:00 PM (peak usage)
spike_duration = 120  # 2 hours
legitimate_spike = normal_traffic.copy()
for i in range(spike_duration):
    # Gradual ramp up and down
    factor = np.sin(np.pi * i / spike_duration)
    legitimate_spike[spike_start + i] += 200 * factor

def analyze_traffic_pattern(traffic, window_size=10):
    """Analyze traffic for anomalies."""
    # Calculate rolling statistics
    rolling_mean = np.convolve(traffic, np.ones(window_size)/window_size, mode='same')
    rolling_std = np.array([np.std(traffic[max(0, i-window_size):i+1]) 
                           for i in range(len(traffic))])
    
    # Detect anomalies (traffic > mean + 3*std)
    threshold = rolling_mean + 3 * rolling_std
    anomalies = traffic > threshold
    
    # Calculate rate of change
    rate_of_change = np.diff(traffic, prepend=traffic[0])
    
    return {
        'rolling_mean': rolling_mean,
        'threshold': threshold,
        'anomalies': anomalies,
        'rate_of_change': rate_of_change
    }

# Analyze patterns
normal_analysis = analyze_traffic_pattern(normal_traffic)
ddos_analysis = analyze_traffic_pattern(ddos_traffic)
spike_analysis = analyze_traffic_pattern(legitimate_spike)

# Create visualizations
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('DDoS Traffic Pattern Analysis', fontsize=16, fontweight='bold')

# Convert minutes to hours for x-axis
hours = minutes / 60

# Plot 1: Normal Traffic
axes[0, 0].plot(hours, normal_traffic, label='Traffic', alpha=0.7)
axes[0, 0].plot(hours, normal_analysis['rolling_mean'], label='Mean', linewidth=2)
axes[0, 0].plot(hours, normal_analysis['threshold'], label='Threshold', linestyle='--')
axes[0, 0].fill_between(hours, 0, normal_analysis['anomalies']*max(normal_traffic), 
                        alpha=0.3, color='red', label='Anomalies')
axes[0, 0].set_title('Normal Traffic Pattern')
axes[0, 0].set_xlabel('Hour of Day')
axes[0, 0].set_ylabel('Requests/min')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Normal Traffic Rate of Change
axes[0, 1].plot(hours, normal_analysis['rate_of_change'])
axes[0, 1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[0, 1].set_title('Normal Traffic - Rate of Change')
axes[0, 1].set_xlabel('Hour of Day')
axes[0, 1].set_ylabel('Change (req/min)')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: DDoS Attack
axes[1, 0].plot(hours, ddos_traffic, label='Traffic', alpha=0.7)
axes[1, 0].plot(hours, ddos_analysis['rolling_mean'], label='Mean', linewidth=2)
axes[1, 0].plot(hours, ddos_analysis['threshold'], label='Threshold', linestyle='--')
axes[1, 0].fill_between(hours, 0, ddos_analysis['anomalies']*max(ddos_traffic), 
                        alpha=0.3, color='red', label='ANOMALIES DETECTED')
axes[1, 0].set_title('DDoS Attack Pattern (Sudden Spike)')
axes[1, 0].set_xlabel('Hour of Day')
axes[1, 0].set_ylabel('Requests/min')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: DDoS Rate of Change
axes[1, 1].plot(hours, ddos_analysis['rate_of_change'], color='red')
axes[1, 1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[1, 1].set_title('DDoS Attack - Rate of Change (Sudden Jump)')
axes[1, 1].set_xlabel('Hour of Day')
axes[1, 1].set_ylabel('Change (req/min)')
axes[1, 1].grid(True, alpha=0.3)

# Plot 5: Legitimate Spike
axes[2, 0].plot(hours, legitimate_spike, label='Traffic', alpha=0.7, color='green')
axes[2, 0].plot(hours, spike_analysis['rolling_mean'], label='Mean', linewidth=2)
axes[2, 0].plot(hours, spike_analysis['threshold'], label='Threshold', linestyle='--')
axes[2, 0].fill_between(hours, 0, spike_analysis['anomalies']*max(legitimate_spike), 
                        alpha=0.3, color='orange', label='Minor Anomalies')
axes[2, 0].set_title('Legitimate Traffic Spike (Gradual)')
axes[2, 0].set_xlabel('Hour of Day')
axes[2, 0].set_ylabel('Requests/min')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

# Plot 6: Legitimate Spike Rate of Change
axes[2, 1].plot(hours, spike_analysis['rate_of_change'], color='green')
axes[2, 1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[2, 1].set_title('Legitimate Spike - Rate of Change (Gradual)')
axes[2, 1].set_xlabel('Hour of Day')
axes[2, 1].set_ylabel('Change (req/min)')
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print analysis summary
print("\nTraffic Pattern Analysis Summary")
print("=" * 80)

def summarize_pattern(name, traffic, analysis):
    anomaly_count = np.sum(analysis['anomalies'])
    max_rate_change = np.max(np.abs(analysis['rate_of_change']))
    avg_traffic = np.mean(traffic)
    max_traffic = np.max(traffic)
    
    print(f"\n{name}:")
    print(f"  Average Traffic: {avg_traffic:.1f} req/min")
    print(f"  Peak Traffic: {max_traffic:.1f} req/min")
    print(f"  Anomaly Minutes: {anomaly_count} ({anomaly_count/len(traffic)*100:.1f}%)")
    print(f"  Max Rate Change: {max_rate_change:.1f} req/min")

summarize_pattern("Normal Traffic", normal_traffic, normal_analysis)
summarize_pattern("DDoS Attack", ddos_traffic, ddos_analysis)
summarize_pattern("Legitimate Spike", legitimate_spike, spike_analysis)

print("\n" + "=" * 80)
print("\nKEY DIFFERENCES:")
print("-" * 80)
print("""
DDoS Attack Indicators:
  - Sudden, extreme spike (500+ req/min increase)
  - Very high rate of change
  - Doesn't follow normal daily patterns
  - Sharp drop-off when attack ends
  
Legitimate Spike Indicators:
  - Gradual increase and decrease
  - Follows predictable patterns (peak hours)
  - Lower rate of change
  - Correlates with known events (product launches, sales)
""")

**Traffic Analysis Techniques:**
1. **Baseline Modeling**: Understand normal traffic patterns
2. **Anomaly Detection**: Statistical outliers (mean + 3σ)
3. **Rate of Change**: Sudden spikes indicate attacks
4. **Time-Series Analysis**: Compare to historical patterns
5. **Entropy Analysis**: DDoS often has low entropy (repetitive)
6. **Geo-Distribution**: Unexpected source countries
7. **Protocol Analysis**: Invalid or malformed packets

## Lab 7: CDN and Cloud DDoS Protection Configuration

Configure modern cloud-based DDoS protection services (Cloudflare-style examples).

In [ ]:
# Examples of CDN/Cloud DDoS protection configurations
# These are conceptual examples based on common CDN features

cloudflare_style_config = {
    'firewall_rules': [
        {
            'name': 'Rate Limit API Endpoints',
            'expression': '(http.request.uri.path contains "/api/")',
            'action': 'challenge',
            'rate_limit': '100 requests per 10 seconds',
            'description': 'Challenge requests exceeding 100/10s to API'
        },
        {
            'name': 'Block Known Attack IPs',
            'expression': '(ip.src in $threat_score > 50)',
            'action': 'block',
            'description': 'Block IPs with high threat scores'
        },
        {
            'name': 'Challenge Suspicious User Agents',
            'expression': '(http.user_agent contains "curl" or http.user_agent contains "python")',
            'action': 'js_challenge',
            'description': 'JavaScript challenge for automated tools'
        },
        {
            'name': 'Geo-Block High-Risk Countries',
            'expression': '(ip.geoip.country in {"XX" "YY"})',
            'action': 'challenge',
            'description': 'Challenge traffic from specified countries'
        },
        {
            'name': 'Protect Login Endpoints',
            'expression': '(http.request.uri.path eq "/login" and http.request.method eq "POST")',
            'action': 'challenge',
            'rate_limit': '5 requests per minute',
            'description': 'Strict rate limit on login attempts'
        }
    ],
    'ddos_protection': {
        'level': 'high',
        'sensitivity': 'medium',
        'features': [
            'SYN Flood Protection',
            'UDP Flood Protection',
            'ICMP Flood Protection',
            'DNS Amplification Protection',
            'HTTP Flood Protection',
            'Slowloris Protection'
        ]
    },
    'bot_management': {
        'enabled': True,
        'modes': {
            'good_bots': 'allow',  # Search engines, monitoring
            'bad_bots': 'block',   # Scrapers, attackers
            'unknown_bots': 'challenge'  # Verify with JS/CAPTCHA
        }
    },
    'caching_rules': [
        {
            'pattern': '/static/*',
            'cache_level': 'aggressive',
            'edge_ttl': 86400,  # 1 day
            'description': 'Cache static assets aggressively'
        },
        {
            'pattern': '/api/*',
            'cache_level': 'bypass',
            'description': 'Bypass cache for API calls'
        }
    ],
    'security_headers': {
        'X-Frame-Options': 'DENY',
        'X-Content-Type-Options': 'nosniff',
        'X-XSS-Protection': '1; mode=block',
        'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload',
        'Content-Security-Policy': "default-src 'self'; script-src 'self' 'unsafe-inline'",
        'Referrer-Policy': 'strict-origin-when-cross-origin'
    }
}

aws_shield_style_config = {
    'protection_level': 'Advanced',
    'features': [
        'Layer 3/4 DDoS Protection',
        'Layer 7 DDoS Protection',
        'Real-time Attack Visibility',
        '24/7 DDoS Response Team',
        'Cost Protection (no surge charges)'
    ],
    'rate_based_rules': [
        {
            'name': 'RateLimitGeneral',
            'rate_limit': 2000,
            'rate_key': 'IP',
            'action': 'BLOCK',
            'aggregation_period': 5  # minutes
        },
        {
            'name': 'RateLimitAPI',
            'rate_limit': 100,
            'rate_key': 'IP',
            'action': 'COUNT',  # Monitor first
            'scope': 'URI',
            'uri_pattern': '/api/*'
        }
    ],
    'health_checks': [
        {
            'protocol': 'HTTPS',
            'path': '/health',
            'interval': 30,
            'timeout': 5,
            'unhealthy_threshold': 2
        }
    ]
}

print("CDN/Cloud DDoS Protection Configuration Guide")
print("=" * 80)

print("\nCLOUDFLARE-STYLE CONFIGURATION")
print("-" * 80)

print("\nFirewall Rules:")
for rule in cloudflare_style_config['firewall_rules']:
    print(f"\n  Rule: {rule['name']}")
    print(f"  Expression: {rule['expression']}")
    print(f"  Action: {rule['action'].upper()}")
    if 'rate_limit' in rule:
        print(f"  Rate Limit: {rule['rate_limit']}")
    print(f"  Description: {rule['description']}")

print("\n\nDDoS Protection Settings:")
print(f"  Level: {cloudflare_style_config['ddos_protection']['level'].upper()}")
print(f"  Sensitivity: {cloudflare_style_config['ddos_protection']['sensitivity']}")
print("\n  Protected Attack Types:")
for feature in cloudflare_style_config['ddos_protection']['features']:
    print(f"    - {feature}")

print("\n\nBot Management:")
print(f"  Enabled: {cloudflare_style_config['bot_management']['enabled']}")
print("  Bot Policies:")
for bot_type, action in cloudflare_style_config['bot_management']['modes'].items():
    print(f"    {bot_type}: {action.upper()}")

print("\n\nSecurity Headers:")
for header, value in cloudflare_style_config['security_headers'].items():
    print(f"  {header}: {value}")

print("\n" + "=" * 80)
print("AWS SHIELD ADVANCED-STYLE CONFIGURATION")
print("-" * 80)

print(f"\nProtection Level: {aws_shield_style_config['protection_level']}")
print("\nFeatures:")
for feature in aws_shield_style_config['features']:
    print(f"  - {feature}")

print("\nRate-Based Rules:")
for rule in aws_shield_style_config['rate_based_rules']:
    print(f"\n  Rule: {rule['name']}")
    print(f"  Rate Limit: {rule['rate_limit']} requests per {rule.get('aggregation_period', 5)} min")
    print(f"  Key: {rule['rate_key']}")
    print(f"  Action: {rule['action']}")
    if 'uri_pattern' in rule:
        print(f"  Scope: {rule['uri_pattern']}")

print("\n" + "=" * 80)
print("\nDEPLOYMENT CHECKLIST")
print("-" * 80)
print("""
1. DNS Configuration:
   - Point domain to CDN (change A/CNAME records)
   - Enable CDN proxy (orange cloud in Cloudflare)
   - Configure SSL/TLS (Full/Strict mode)

2. Initial Protection:
   - Enable DDoS protection (Layer 3/4 automatic)
   - Configure rate limiting rules
   - Set up bot management
   - Enable security headers

3. Fine-Tuning:
   - Monitor false positives
   - Adjust rate limits based on legitimate traffic
   - Whitelist known good IPs/services
   - Configure custom error pages

4. Monitoring:
   - Set up alerts for attacks
   - Review analytics dashboard daily
   - Check firewall event logs
   - Monitor origin server load

5. Testing:
   - Test firewall rules (don't lock yourself out!)
   - Verify rate limits work as expected
   - Confirm legitimate traffic passes through
   - Test failover scenarios
""")

print("\n" + "=" * 80)
print("COST CONSIDERATIONS")
print("-" * 80)
print("""
Cloudflare:
  - Free tier: Basic DDoS protection
  - Pro ($20/mo): Advanced DDoS + WAF
  - Business ($200/mo): 100% uptime SLA
  - Enterprise (custom): Dedicated support, custom rules

AWS Shield:
  - Standard: Free (automatic Layer 3/4)
  - Advanced: $3,000/mo + data transfer fees
    Benefits: Layer 7 protection, cost protection, 24/7 DRT

Akamai:
  - Enterprise pricing (contact sales)
  - Handles largest attacks (multiple Tbps)
  - Premium global network
""")

## Summary and Key Takeaways

**Labs Completed:**
1. Token bucket rate limiting implementation
2. HTTP flood detection from access logs
3. DNS amplification factor calculation
4. SYN flood defense configuration (Linux kernel + iptables)
5. Nginx application-layer rate limiting
6. DDoS traffic pattern analysis and visualization
7. Modern CDN/cloud DDoS protection setup

**Defense-in-Depth Strategy:**
- **Network Layer**: SYN cookies, firewall rules, BCP38
- **Transport Layer**: Connection limits, timeout tuning
- **Application Layer**: Rate limiting, bot detection, caching
- **Cloud/CDN**: Anycast, global distribution, massive bandwidth
- **Monitoring**: Real-time traffic analysis, anomaly detection

**Best Practices:**
1. Implement multiple layers of defense
2. Use cloud-based DDoS mitigation for large attacks
3. Monitor normal traffic patterns to detect anomalies
4. Rate limit aggressively on sensitive endpoints
5. Keep systems updated and properly configured
6. Have an incident response plan
7. Test defenses regularly

**Remember:** Modern DDoS attacks can reach multiple Terabits per second. No single organization can defend against such attacks alone—cloud-based mitigation services are essential for serious protection.

**Continue Learning:**
- Return to Module #18 in HackLearn Pro
- Complete the challenge quiz
- Explore the Tools and References tabs
- Practice in authorized lab environments only